In [1]:
import requests
import json
import re
import time
import pandas as pd


def fetch_sse_data(mdate='202403'):
    """
    抓取上海证券交易所指定月份的数据。

    Args:
        mdate (str): 查询的月份，格式为 'YYYYMM'，例如 '202403'。

    Returns:
        dict: 解析后的数据字典，如果抓取或解析失败则返回 None。
    """
    # 1. 定义请求 URL
    url = 'https://query.sse.com.cn//commonQuery.do'

    # 2. 定义请求参数
    # jsonCallBack 和 _ (时间戳) 通常是动态生成的，
    # 为了模拟浏览器行为，我们可以生成一个随机的 jsonCallBack 后缀和当前时间戳。
    timestamp_ms = str(int(time.time() * 1000))
    json_callback_name = f'jsonpCallback{timestamp_ms}'

    params = {
        'jsonCallBack': json_callback_name,
        'sqlId': 'COMMON_SSE_TZZ_M_ALL_ACCT_C',
        'isPagination': 'false',
        'MDATE': mdate, # 查询的月份
        '_': timestamp_ms # 当前时间戳，防止缓存
    }

    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:142.0) Gecko/20100101 Firefox/142.0',
        'Accept': '*/*',
        'Accept-Language': 'zh-CN,zh;q=0.8,zh-TW;q=0.7,zh-HK;q=0.5,en-US;q=0.3,en;q=0.2',
        'Accept-Encoding': 'gzip, deflate, br, zstd',
        'Referer': 'https://www.sse.com.cn/',
        'Connection': 'keep-alive',
        'Sec-Fetch-Dest': 'script',
        'Sec-Fetch-Mode': 'no-cors',
        'Sec-Fetch-Site': 'same-site',
        'Pragma': 'no-cache',
        'Cache-Control': 'no-cache'
    }

    # 4. 发送 GET 请求
    try:
        response = requests.get(url, headers=headers, params=params, timeout=15)
        response.raise_for_status()

        jsonp_text = response.text

        pattern = rf'{re.escape(json_callback_name)}\((.*)\)'
        match = re.match(pattern, jsonp_text, re.DOTALL) # re.DOTALL 使 . 匹配包括换行符在内的所有字符

        if match:
            json_string = match.group(1)
            data = json.loads(json_string)
            return data
        else:
            raise ValueError(jsonp_text[:500])

    except Exception as e:
        print(f"发生错误：{e}")
        return None


def convert_data_to_dataframe(data) -> pd.DataFrame:
    df = pd.DataFrame(data['result'])
    df['TERM'] = pd.to_datetime(df['TERM'], format='%Y.%m', errors='coerce')
    numeric_cols = ['TOTAL', 'A_ACCT', 'B_ACCT', 'FUND_ACCT']
    for col in numeric_cols:
        # 使用 errors='coerce' 将无法解析的值转换为 NaN
        df[col] = pd.to_numeric(df[col], errors='coerce')
    df.dropna(subset=['TERM'], inplace=True)

    new_column_names = {
        'TOTAL': '总户数',
        'A_ACCT': 'A股账户数',
        'B_ACCT': 'B股账户数',
        'MDATE': '查询月份',
        'TERM': '统计期',
        'FUND_ACCT': '基金账户数'
    }
    df.rename(columns=new_column_names, inplace=True)
    return df



data_202312 = fetch_sse_data(mdate='202312')
df_202312 = convert_data_to_dataframe(data_202312)
data_202412 = fetch_sse_data(mdate='202412')
df_202412 = convert_data_to_dataframe(data_202412)
data_202507 = fetch_sse_data(mdate='202507')
df_202507 = convert_data_to_dataframe(data_202507)

df = pd.concat([df_202312, df_202412, df_202507], axis=0, ignore_index=True)
df

,总户数,A股账户数,查询月份,B股账户数,统计期,基金账户数
0,164.69,152.88,202312,0.07,2023-01-01,11.74
1,319.23,295.05,202312,0.07,2023-02-01,24.11
2,363.52,333.35,202312,0.05,2023-03-01,30.11
3,197.72,180.95,202312,0.03,2023-04-01,16.73
4,189.39,171.84,202312,0.05,2023-05-01,17.51
5,194.11,174.75,202312,0.03,2023-06-01,19.32
6,179.55,162.12,202312,0.03,2023-07-01,17.40
7,190.67,174.29,202312,0.03,2023-08-01,16.35
8,146.99,133.84,202312,0.05,2023-09-01,13.10
9,129.57,117.33,202312,0.03,2023-10-01,12.21
